In [2]:
import numpy as np
import random
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import sys
from PIL import Image
sys.modules['Image'] = Image 
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from PIL import Image
print(Image.__file__)
import Image
print(Image.__file__)

C:\MyPrograms\anaconda332\envs\tensorflow\lib\site-packages\PIL\Image.py
C:\MyPrograms\anaconda332\envs\tensorflow\lib\site-packages\PIL\Image.py


Using TensorFlow backend.


In [3]:
#verifying that the images counts have been correctly retrieved for testing and training data for horses and humans
train_cats_dir = os.path.join('cats_vs_dogs_full/train/cats')
test_cats_dir = os.path.join('cats_vs_dogs_full/test/cats')

train_dogs_dir = os.path.join('cats_vs_dogs_full/train/dogs')
test_dogs_dir = os.path.join('cats_vs_dogs_full/test/dogs')

print('total training cat images:', len(os.listdir(train_cats_dir)))
print('total training dog images:', len(os.listdir(train_dogs_dir)))

print('\ntotal testing cat images:', len(os.listdir(test_cats_dir)))
print('total testing dog images:', len(os.listdir(test_dogs_dir)))

total training cat images: 7000
total training dog images: 7000

total testing cat images: 700
total testing dog images: 700


In [4]:
#images are off diff sizes
urls = ["cats_vs_dogs_full/train/cats/cat.10001.jpg", "cats_vs_dogs_full/train/cats/cat.10002.jpg",
        "cats_vs_dogs_full/train/cats/cat.10003.jpg","cats_vs_dogs_full/train/cats/cat.10004.jpg"]

for i in range(4):
    current_url = urls[i]
    img = image.load_img(current_url, target_size=(150, 150))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    img = np.vstack([x])
    show_img = mpimg.imread(current_url)
    plt.imshow(show_img)

FileNotFoundError: [Errno 2] No such file or directory: 'cats_vs_dogs_full/train/cats/cat.10001.jpg'

In [5]:
train_data = ImageDataGenerator(rescale = 1./255)
test_data = ImageDataGenerator(rescale = 1./255)

train_dir = "cats_vs_dogs_full/train"
test_dir = "cats_vs_dogs_full/test"

train_generator = train_data.flow_from_directory(
    train_dir, 
    target_size = (300, 300), 
    batch_size = 700, 
    class_mode = 'binary' 
)

test_generator = test_data.flow_from_directory(
    test_dir, 
    target_size = (300, 300), 
    batch_size = 70, 
    class_mode = 'binary' 
)

Found 14000 images belonging to 2 classes.
Found 1400 images belonging to 2 classes.


In [6]:
#building the model
model = tf.keras.models.Sequential([
    #conv net part
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)), #3 is for rgb color channels
    tf.keras.layers.MaxPooling2D(2, 2),
    
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    # The third convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    #dnn part
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9248)              0

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr = 0.0001),
              metrics=['accuracy'])

#training model after loading data in using an image generator
history = model.fit (
    #training part
    train_generator,
    steps_per_epoch = 10, 
    epochs = 15,
    
    #testing part
    validation_data = test_generator,
    validation_steps = 10,
    verbose = 1
)

Epoch 1/15
